<h1><center>Real World Data Science: <br>Visualizing New York Crime Data</h1></center>

<img src="https://static-assets.ny.gov/sites/all/themes/ny_gov/images/nygov-logo.png">


## Step 1: Gather Data
<hr>

In [ ]:
# Load the dataset from the NY Open Data Portal 
# Using the API (2010-Current year data)

# make sure to install these packages before running:
# pip3 install pandas
# pip3 install sodapy

import pandas as pd
import numpy as np
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.ny.gov", None)

# First 200k results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

### Pro-tip #1 - Limit your download from the source for large datasets ###
### Download only a subset from 2011-Current (best practice for bandwidth and processing to download only data needed)
### A more complex clause here could limit columns, but typically we want to preserve variables/featurs/columns

results = client.get("ca8h-8gjq.", where='year >2010', limit=200000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
### Pro-tip #2 After you download data save a copy in case it changes at the source and to save future bandwidth
### Compress csv files to further save space! 

compression_opts = dict(method='zip', archive_name='nys_total_crime.csv')  
results_df.to_csv('nys_total_crime.zip', index=False, compression=compression_opts)  

# Un-comment code below to restore the saved data
#results_df = pd.read_csv('nys_total_crime.zip', compression='zip', header=0, sep=',', quotechar='"')

## Step 2: Verify the Import
<hr>

In [ ]:
### Pro-tip #3 Check the number of rows and columns matches what was sent / downloaded
# 7099,15 as of 10-26-2022
results_df.shape

In [ ]:
# What does the dataframe look like? 
results_df.dtypes

In [ ]:
# Check for Null/NaN values
results_df.isnull().sum()

## Step 3: Data Hygiene and Transforms
<hr>

In [ ]:
### Pro-tip #4 Make sure the datatypes are correct for data you will be analyzing, keep only the data you need

# Sometimes data-types are messy when importing data, below is code so you can fix this if the import isn't OK
# Regardless of the original source/method
# Ensure data types are correct for analysis

results_df['county'] = results_df['county'].astype(str)
results_df['agency'] = results_df['agency'].astype(str)
results_df['year'] = results_df['year'].astype(int)
results_df['total_index_crimes'] = results_df['total_index_crimes'].astype(int)
results_df['violent'] = results_df['violent'].astype(int)
results_df['property'] = results_df['property'].astype(int)

In [ ]:
# Keep only certain columns - Saves on memory, processing, and disk/cloud size
results_df = results_df[['county', 'agency', 'year', 'total_index_crimes', 'violent', 'property']]

In [ ]:
# Re-check to make sure they are correct!
results_df.dtypes

In [ ]:
# Roll-up to county-year level and rename column headings for rolled-up dataframes
# I am rolling these up one at a time to show a merge later! 
# Analysis of these other variables is not shown in the interest of time

crimes_total_df=results_df.groupby(["county", "year"], as_index=False)["total_index_crimes"].sum()
crimes_total_df.columns = ['county', 'year', 'index_total']

# always check every intermediary step! 
crimes_total_df.head()

In [ ]:
violent_crimes_df=results_df.groupby(["county", "year"], as_index=False)["violent"].sum()
violent_crimes_df.columns = ['county', 'year', 'violent_total']

violent_crimes_df.head()

In [ ]:
property_crimes_df=results_df.groupby(["county", "year"], as_index=False)["property"].sum()
property_crimes_df.columns = ['county', 'year', 'property_total']

property_crimes_df.head()

In [ ]:
# This merge is unnecessary in the context of this presentation; however, it is included here 
# for use in a future project. 

# Merge The Datasets Length-Wise
from functools import reduce

#define list of DataFrames
dfs = [crimes_total_df, violent_crimes_df, property_crimes_df]

#merge all DataFrames into one
county_year_df = reduce(lambda  left,right: pd.merge(left,right,on=['county', 'year'],
                                            how='outer'), dfs)

#Use the join type that is appropriate for your data, check the documentation! 

# As always, check to make sure you have the data format/results you are expecting
county_year_df.head()

In [ ]:
### Pro-tip #5: Check to make sure there are no Null and NaN that could mess up calculations later
county_year_df.isnull().sum()

In [ ]:
### Pro-tip #6: Delete temporary / old datasets 

# The Combined Data looks good, so let's delete our old data to save memory this makes a big difference on larger data
del results_df
del crimes_total_df 
del violent_crimes_df 
del property_crimes_df

In [ ]:
# Calculate Z Scores by County to Normalize County Level Differences
# This will help us look at county level trends and not comparisons across counties with different size and demographics
# Another approach to this would be to get population levels and look at crimes per capita

from scipy.stats import zscore

county_year_df['index_Z']=county_year_df.groupby(['county']).index_total.transform(lambda x : zscore(x,ddof=1))
county_year_df['violent_Z']=county_year_df.groupby(['county']).violent_total.transform(lambda x : zscore(x,ddof=1))
county_year_df['property_Z']=county_year_df.groupby(['county']).property_total.transform(lambda x : zscore(x,ddof=1))

county_year_df.head()

## Step 4: Data Analysis
<hr>

In [ ]:
# Pivot Tables help you to "see" your data.

# Overall Crime Index, you can see which years were the worst in each county
# Since these are Z-scores, they are standard deviations above or below the mean. (higher is worse)

pivot = np.round(pd.pivot_table(county_year_df, values='index_Z', 
                                index='county', 
                                columns='year', 
                                aggfunc=np.sum), decimals=2)
pivot

# Although We can see a trend in the pivot-data, having a chart will 
# help to tell the story

# The pivot table is a good way to get a peak into the data
# For the purpose of this walkthrough, we will only look at total_index

In [ ]:
# Sometimes you may need interactive tables.  This is the code for those that prefer an interactive Pivot Table
# Un-Comment 2 lines below for Interactive Pivot Tables

#from pivottablejs import pivot_ui
#pivot_ui(county_year_df, outfile_path='pivottablejs.html')


# Step 5: Data Visualizations
<hr>

In [ ]:
# Let's Distribution of Index_Z by County in a box-plot
# The pivot table indiactes a pattern, but what is it? 

import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(y=county_year_df["index_Z"], x=county_year_df["county"] ).set(ylabel='Total Crime Z-Scores');
plt.show()

In [ ]:
# Seaborn can produce some nice graphs; however, plotly is interactive
# and the interactivity can help you determine datapoints of interest (outliers)

import plotly.express as px

fig = px.box(county_year_df, y="index_Z", x="county")
fig.show()

In [ ]:
### Pro-tip #7 Leave comments and interpretations/findings so you can
### follow your train of thought after days/weeks/months/years
### headings help the finding stand out

### <center><u>Finding #1</u></center>

Bronx & Otsego counties have years with total crime spikes 2.45 and 2.16 higher than the standard deviation. 

Each county is consistent internally with the distribution of Z-scores for their crimes. <hr>

In [ ]:
# Let's Distribution of Index_Z by County in a box-plot
# Seaborn makes decent univariate plots that are a good starting point for data analysis

sns.boxplot( y=county_year_df["index_Z"], x=county_year_df["year"] ).set(xlabel='Year', ylabel='Total Crime Z-Scores');
plt.show()

In [ ]:
fig = px.box(county_year_df, y="index_Z", x="year")
fig.show()

### <center><u>Finding #2</u></center>
Over time, the Crime Index appears to decrease for most counties; however, some counties have large spikes in 2021and occur despite the overall trend.<hr>


In [ ]:
# County Map Chlorplast

# Add a FIPS code to our data so we can plot with county map in New York
# This could be done other ways; however, I want to highlight a case-type statement as a way to create 
# A new variable in a dataset that wasn't a basic calculation. Notice the crazy amount of ) at the end
# Fips codes can be found: https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt#:~:text=FIPS%20codes%20are%20numbers%20which,to%20which%20the%20county%20belongs.


### Pro-tip #8 - "numbers" that are identifies should be stored as strings

# In this case I have assigned fips code to be a string instead of a number, because like zipcode and areacode
# it is an identifier and not a literal value where the numeric increase indicates a scalar difference

county_year_df['fips'] = np.where(county_year_df['county'] == 'Albany', '36001', 
                              np.where(county_year_df['county'] == 'Allegany', '36003',
                              np.where(county_year_df['county'] == 'Bronx', '36005',
                              np.where(county_year_df['county'] == 'Broome', '36007',
                              np.where(county_year_df['county'] == 'Cattaraugus', '36009',
                              np.where(county_year_df['county'] == 'Cayuga', '36011',
                              np.where(county_year_df['county'] == 'Chautauqua', '36013',
                              np.where(county_year_df['county'] == 'Chemung', '36015',     
                              np.where(county_year_df['county'] == 'Chenango', '36017',         
                              np.where(county_year_df['county'] == 'Clinton', '36019', 
                              np.where(county_year_df['county'] == 'Columbua', '36021',         
                              np.where(county_year_df['county'] == 'Cortland', '36023',         
                              np.where(county_year_df['county'] == 'Delaware', '36025',         
                              np.where(county_year_df['county'] == 'Dutchess', '36027',         
                              np.where(county_year_df['county'] == 'Erie', '36029', 
                              np.where(county_year_df['county'] == 'Essex', '36031',         
                              np.where(county_year_df['county'] == 'Franklin', '36033',         
                              np.where(county_year_df['county'] == 'Fulton', '36035',         
                              np.where(county_year_df['county'] == 'Genesee', '36037',    
                              np.where(county_year_df['county'] == 'Greene', '36039',         
                              np.where(county_year_df['county'] == 'Hamilton', '36041',         
                              np.where(county_year_df['county'] == 'Herkimer', '36043',         
                              np.where(county_year_df['county'] == 'Jefferson', '36045',         
                              np.where(county_year_df['county'] == 'Kings', '36047',         
                              np.where(county_year_df['county'] == 'Lewis', '36049',         
                              np.where(county_year_df['county'] == 'Livingston', '36051',         
                              np.where(county_year_df['county'] == 'Madison', '36053',         
                              np.where(county_year_df['county'] == 'Monroe', '36055',         
                              np.where(county_year_df['county'] == 'Montgomery', '36057', 
                              np.where(county_year_df['county'] == 'Nassau', '36059',         
                              np.where(county_year_df['county'] == 'New York', '36061',         
                              np.where(county_year_df['county'] == 'Niagara', '36063',        
                              np.where(county_year_df['county'] == 'Oneida', '36065',         
                              np.where(county_year_df['county'] == 'Onondaga', '36067',         
                              np.where(county_year_df['county'] == 'Ontario', '36069',         
                              np.where(county_year_df['county'] == 'Orange', '36071',         
                              np.where(county_year_df['county'] == 'Orleans', '36073',         
                              np.where(county_year_df['county'] == 'Oswego', '36075',         
                              np.where(county_year_df['county'] == 'Otsego', '36077', 
                              np.where(county_year_df['county'] == 'Putnam', '36079',         
                              np.where(county_year_df['county'] == 'Queens', '36081',         
                              np.where(county_year_df['county'] == 'Rensselaer', '36083',         
                              np.where(county_year_df['county'] == 'Richmond', '36085', 
                              np.where(county_year_df['county'] == 'Rockland', '36087',         
                              np.where(county_year_df['county'] == 'St Lawrence', '36089',         
                              np.where(county_year_df['county'] == 'Saratoga', '36091',         
                              np.where(county_year_df['county'] == 'Schenectady', '36093',         
                              np.where(county_year_df['county'] == 'Schoharie', '36095',         
                              np.where(county_year_df['county'] == 'Schuyler', '36097',         
                              np.where(county_year_df['county'] == 'Seneca', '36099',         
                              np.where(county_year_df['county'] == 'Steuben', '36101',         
                              np.where(county_year_df['county'] == 'Suffolk', '36103',         
                              np.where(county_year_df['county'] == 'Sullivan', '36105',         
                              np.where(county_year_df['county'] == 'Tioga', '36107',         
                              np.where(county_year_df['county'] == 'Tompkins', '36109',   
                              np.where(county_year_df['county'] == 'Ulster', '36111',          
                              np.where(county_year_df['county'] == 'Warren', '36113',         
                              np.where(county_year_df['county'] == 'Washington', '36115',         
                              np.where(county_year_df['county'] == 'Wayne', '36117',   
                              np.where(county_year_df['county'] == 'Westchester', '36119',         
                              np.where(county_year_df['county'] == 'Wyoming', '36121',         
                              np.where(county_year_df['county'] == 'Yates', '36123', ''))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))         

### Pro-tip #9 - Strategically assign NaN or '' to ensure manual coding worked


In [ ]:
# Verify a 1-1 relationship of county and fips
county_year_df.groupby(['county', 'fips']).count()

In [ ]:
# Verify no NaNs
county_year_df.isnull().sum()

In [ ]:
# Visualize with a slider to adjust view over time

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd

#vis_df = county_year_df.loc[county_year_df['year'] == 2021, ['fips', 'county', 'index_Z']]

import plotly.express as px

fig = px.choropleth(county_year_df, geojson=counties, locations='fips', color='index_Z',
                           color_continuous_scale="Viridis",
                           range_color=(-3, 3),
                           animation_frame='year',
                           animation_group='fips',
                           hover_name='county',
                           scope="usa",
                           labels={'index_Z':'County Crime Index'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout({'title' : {'text' : 'New York State Total Crime Z-Scores', 'x' : 0.5, 'y':0.9}})
fig.update_layout()
fig.show()


In [ ]:
### Pro-tip 10 - Always consider your audience/viewer when presenting 
### numbers/charts/graphs.

# Most end consumers don't understand z-scores to non-standard scaling 
# Let's adjust or z-score -> 0-100 to make it easier to explain/understand

index_min = county_year_df['index_Z'].min()
index_max = county_year_df['index_Z'].max()

### Pro-tip 11 - Explain forumlas in comments, complex formulas can be
### challenging to remember / explain days/months/years later or 
### for someone else maintaining your code after you leave. 

# Add the absolute value of minimum to bring new minimum to zero
# Then divide by the max + abs(min) to make it 0-1
# Multiple by 100 and round to nearest whole number
county_year_df['index_Z_scaled'] = ((county_year_df['index_Z'] + abs(index_min)) / (index_max + abs(index_min)) * 100).round(decimals=0)

In [ ]:
# Check and make sure the new variable does run from 0-100
sns.set(style="darkgrid")

sns.histplot(data=county_year_df, x="index_Z_scaled", bins=10).set(xlabel='Deciles', ylabel='Z-Scores Scaled');
plt.show()

In [ ]:
# Finally, re-graph the map
fig = px.choropleth(county_year_df, geojson=counties, locations='fips', color='index_Z_scaled',
                           color_continuous_scale="Viridis",
                           range_color=(0, 100),
                           animation_frame='year',
                           animation_group='fips',
                           hover_name='county',
                           scope="usa",
                           labels={'index_Z':'County Crime Index'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout({'title' : {'text' : 'New York State Total Crime Z-Scores Scaled', 'x' : 0.2, 'y':0.9}})
fig.update_layout()
fig.show()

### <center><u>Finding #3</u></center>
As a whole, total crime decreases across the state; however, severe and large spikes happen in NYC/Bronx/Kings/Queens in the year 2021. 
<hr>

In [ ]:
### Pro-tip 12: Consistently utilize headers and formating 
### to make your report easy to read and include conclusions / about sections 
### break up code blocks to make processes easier to see/follow

# <u>Data Conclusions</u>
<ol>
    <li>The number of total crimes in counties were relatively consistent year over year.</li>
    <li>Total crimes generally decreased from 2010-2021.</li>
    <li>Crime spikes in and around the New York City metro-area in 2021 were relatively large especially compared to the overall decreasing trend.</li> 
</ol>

# <u>Recommendations / Next Steps</u>
<ol>
    <li>Analysis of the data using per-capita instead of Z-scores may be helpful in presenting findings.</li>
    <li>Analysis of violent and property crimes and their comparison may be helpful.</li>
    <li>Experimentation with Machine Learning algorithms with other state data may predict increases in crime rates.</li>
</ol>

# <u>About the Author</u>

Samuel Sanfratello is a Data Scientist for Rochester Analytics. He has a Master of Science in Applied Business Analytics from American Public Univserity and is a proud member of the Alumni Advisory Council.<br>


Web: <a href="https://rochesteranalytics.com?utm_source=pycode&utm_medium=about&utm_campaign=nycrimvis"> https://rochesteranalytics.com</a><br>
Twitter: https://twitter.com/RocAnalytics<br>
LinkedIn: https://www.linkedin.com/in/samuel-sanfratello/<br>
LinkedIn (Company Page): https://www.linkedin.com/company/rochester-analytics/
